In [ ]:
import cv2
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Create VideoCapture object and read from USB webcam (usually 0 or 1)
cap = cv2.VideoCapture(0);

# Check if camera opened successfully
if (cap.isOpened() == False):
    print("Error opening video stream or file")

# Set video size to 640 by 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Video size: ", frame_width, "x", frame_height)

# Text on video
font            = cv2.FONT_HERSHEY_SIMPLEX
textOffsetX     = 50
textOffsetY     = 50
fontScale       = 1
fontColor       = (0,0,0)        # Black
lineType        = 2
boxColor        = (255,255,255)  # White

# Global variables
firstTime = True
imageType = 1
filterType = 1
filterSize = 3

# Concatenate images
def concat_tile(im_list_2d):
    return cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in im_list_2d])

# Smoothing filters
def imageFilter(frame, fType, fSize):
    "This applies an image flter to the frame of a video"
            
    if fType == 1:
        fframe = cv2.boxFilter(frame, -1, (fSize,fSize))
        title = 'Average Filter (' + str(fSize) + 'x' + str(fSize) + ')'
    elif fType == 2:
        fframe = cv2.GaussianBlur(frame, (fSize,fSize), 0)
        title = 'Gaussian Filter (' + str(fSize) + 'x' + str(fSize) + ')'
    elif fType == 3:
        fframe = cv2.medianBlur(frame, fSize)
        title = 'Median Filter (' + str(fSize) + 'x' + str(fSize) + ')'
    elif fType == 4:
        fframe = cv2.bilateralFilter(frame, fSize, 75, 75)
        title = 'Bilateral Filter (' + str(fSize) + 'x' + str(fSize) + ')'
    else:
        fframe = frame
    
    # Get the width and height of the text box
    (textWidth, textHeight) = cv2.getTextSize(title, font, fontScale, lineType)[0]
    
    # Make the coords of the box with a small padding of two pixels
    boxCoords = ((textOffsetX, textOffsetY), (textOffsetX+textWidth-2, textOffsetY-textHeight-2))
    cv2.rectangle(fframe, boxCoords[0], boxCoords[1], boxColor, cv2.FILLED)
    
    # Put title on video frame
    cv2.putText(fframe, title, (textOffsetX, textOffsetY), font, fontScale, fontColor, lineType)
    
    return fframe;

# Concatinate all smoothing filters
def imageFilterAll(frame, fSize):
    "This applies all image flters and tiles the videos"
            
    fframe1 = imageFilter(frame, 1, fSize)
    fframe2 = imageFilter(frame, 2, fSize)
    fframe3 = imageFilter(frame, 3, fSize)
    fframe4 = imageFilter(frame, 4, fSize)
    fframe = concat_tile([[fframe1, fframe2],
                          [fframe3, fframe4]])
        
    return fframe;

# Read until video is completed or exit is pressed
while(cap.isOpened()):
    
    # First time or press K on keyboard to get input
    if (firstTime == True) or (cv2.waitKey(20) & 0xFF == ord('k')):
        print('Select color or bw image (1=bw, 2=color): ')
        imageType = int(input())
        print('Select filter (1=average, 2=gaussian, 3=median, 4=bilateral, 0=all): ')
        filterType = int(input())
        print('Filter size (odd number between 3 - 19): ')
        filterSize = int(input())
        firstTime = False
        
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Convert image to grayscale
    if imageType == 1:
        gframe = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    else:
        gframe = frame

    if ret == True:
        # Put the filter code here
        if filterType == 0:
            cframe = imageFilterAll(gframe, filterSize)
        else:
            cframe = imageFilter(gframe, filterType, filterSize)
        
        # Display the resulting frame
        cv2.imshow('Project 1', cframe)

        # Press Q on keyboard to exit
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    else:
        break

# Release the video capture and close all the frames
cap.release()
cv2.destroyAllWindows()